In [1]:
import gymnasium as gym

In [11]:
import time

In [12]:
env = gym.make('CartPole-v1', render_mode='human')

observation, info = env.reset()

print(observation)

total_reward = 0

end_episode = False
while not end_episode:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    print(observation)
    end_episode = terminated or truncated
    time.sleep(1)

env.reset()

env.close()

[ 0.01083597  0.03448986 -0.044824   -0.00105543]
[ 0.01152577 -0.15996155 -0.0448451   0.27715486]
[ 0.00832653  0.03577054 -0.03930201 -0.02932814]
[ 0.00904194  0.23143342 -0.03988857 -0.33414763]
[ 0.01367061  0.03690122 -0.04657152 -0.05430556]
[ 0.01440864  0.23265894 -0.04765764 -0.36131084]
[ 0.01906182  0.42842475 -0.05488385 -0.6686322 ]
[ 0.02763031  0.2341072  -0.0682565  -0.3937225 ]
[ 0.03231246  0.43012798 -0.07613094 -0.70712084]
[ 0.04091502  0.6262174  -0.09027336 -1.022764  ]
[ 0.05343936  0.82241833 -0.11072864 -1.3423712 ]
[ 0.06988773  0.62885016 -0.13757606 -1.0862849 ]
[ 0.08246473  0.43578425 -0.15930176 -0.83974224]
[ 0.09118042  0.24315399 -0.1760966  -0.6010923 ]
[ 0.0960435   0.05087568 -0.18811846 -0.36864042]
[ 0.09706101  0.24810283 -0.19549125 -0.71424305]
[ 0.10202307  0.05614691 -0.20977612 -0.4889027 ]
